In [13]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)
from libs.common import *
from config.config import *

c:\Users\Admin\Data\ads_fraud_detection


# Lựa chọn đặc trưng

### (1) Tạo thư mục lưu trữ (nếu chưa có)

In [14]:
save_dir=f"{exps_dir}/exp1/exp_gan"

os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [15]:
x_train=pd.read_excel(f'{save_dir}/gen_x.xlsx')
y_train=pd.read_excel(f'{save_dir}/gen_y.xlsx')
x_test=pd.read_excel(f'{save_dir}/x_test.xlsx')
y_test=pd.read_excel(f'{save_dir}/y_test.xlsx')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(25, 39) (25, 1) (330, 39) (330, 1)


### (2) Lựa chọn các đặc trưng

In [16]:
x_train

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,day_policy_bind_date,month_policy_bind_date,year_policy_bind_date,day_incident_date,month_incident_date,year_incident_date
0,1,2,2,0,1,0.319438,5,1,2,2,15,1,0.957325,-0.791590,2,0,0,2,6,6,0,0,2,2,0,2,-0.019826,0.612299,-0.455072,-0.069482,8,7,0,-0.886189,0.395797,1.184162,1.043941,-0.947412,0
1,0,2,1,1,1,0.079028,4,1,1,9,10,2,0.753106,-1.131827,0,1,0,0,0,6,0,2,2,2,3,2,0.067456,-0.426662,0.732792,0.016187,3,32,1,0.917827,1.546268,0.633006,-0.465225,-0.947412,0
2,0,2,2,0,1,-0.817344,6,0,5,7,15,3,0.911552,-0.623262,0,1,1,0,0,4,2,2,0,1,3,1,0.023435,0.289158,0.312819,-0.123224,10,2,2,0.805076,-0.179439,1.459740,1.043941,0.953085,0
3,1,0,1,1,1,0.847491,2,1,6,3,9,2,1.013662,-0.343910,0,2,2,1,1,0,0,2,1,0,0,0,-0.081303,-0.389849,-0.370665,0.083232,12,18,2,0.692325,-0.179439,1.046373,1.160031,-0.947412,0
4,0,2,2,0,0,0.596971,4,0,6,7,0,0,1.555900,-0.830986,2,1,1,0,1,4,1,0,2,0,1,1,0.092881,-0.285544,1.016891,-0.058308,12,9,2,-0.998940,-1.042293,0.908584,-1.510033,0.953085,0
5,1,2,0,0,1,0.828201,5,0,2,0,15,5,0.534802,-0.533726,0,2,2,0,1,3,0,2,0,0,1,0,-0.452821,-0.009441,0.012251,-0.635641,10,3,0,0.917827,-1.329911,1.459740,-0.929584,-0.947412,0
6,1,0,0,1,2,1.922199,6,1,1,13,10,1,0.710853,-0.522982,0,0,1,1,6,4,0,2,0,2,0,0,0.096297,-0.602550,0.378697,0.193910,10,3,0,1.368831,-0.754675,1.597528,1.043941,-0.947412,0
7,1,1,1,0,0,0.176931,5,1,6,2,7,4,0.932678,-0.419120,2,0,0,1,0,6,3,0,2,0,0,1,0.621506,1.303577,-0.107153,0.559998,5,14,2,0.241321,-1.042293,0.219640,-1.045674,-0.947412,0
8,1,2,1,2,0,0.010807,6,1,2,4,15,3,0.513676,-0.519400,0,1,0,2,2,0,1,3,0,2,2,0,-0.224749,-0.584143,0.415754,-0.270617,0,33,2,-1.562696,0.395797,-1.571616,-0.116956,0.953085,0
9,1,0,1,0,1,-0.585507,9,1,4,6,10,2,0.291851,-1.063780,0,2,2,2,1,5,0,2,0,0,1,1,-0.224749,-0.584143,0.415754,-0.270617,0,23,0,-0.773438,-1.042293,-0.331516,1.043941,-0.947412,0


In [17]:
def modelcheck_info(model,name):
    print(name)
    try:
        coefficients = model.coef_.flatten()
    except:
        coefficients = model.feature_importances_.flatten()
    feature_importance = pd.DataFrame({
        'Feature': x_train.columns,
        'Coefficient': coefficients
    })

    print('bi loai bo',feature_importance[feature_importance['Coefficient'] == 0])
    feature_importance = feature_importance[feature_importance['Coefficient'] != 0]
    feature_importance['Absolute Coefficient'] = feature_importance['Coefficient'].abs()
    feature_importance.sort_values(by='Absolute Coefficient', ascending=False, inplace=True)

    print("Xếp hạng các đặc trưng theo mức độ quan trọng:")
    print(feature_importance[['Feature', 'Coefficient']])

In [18]:
models=[]
models.append(LogisticRegression())
models.append(LinearDiscriminantAnalysis())
models.append(DecisionTreeClassifier())
names=['LR','LDA','DTC']
feature_SelectFromModel={}
for model,name in zip(models,names):
    check = model.fit(x_train, y_train)
    selector = SelectFromModel(check, prefit=True)

    selected_features = x_train.columns[(selector.get_support())]
    modelcheck_info(model,name)
    print("Các đặc trưng được chọn:")
    feature_SelectFromModel[name]=list(selected_features)
    print(selected_features)

LR
bi loai bo                Feature  Coefficient
38  year_incident_date          0.0
Xếp hạng các đặc trưng theo mức độ quan trọng:
                        Feature  Coefficient
5         policy_annual_premium    -0.782840
30                    auto_make    -0.692171
13                 capital-loss     0.534600
7                   insured_sex     0.525403
23              bodily_injuries     0.513167
2                  policy_state     0.506134
1                           age     0.487866
28               property_claim    -0.449442
4             policy_deductable    -0.383790
3                    policy_csl    -0.380984
17        authorities_contacted    -0.369738
14                incident_type     0.368492
19                incident_city     0.364510
15               collision_type     0.362071
11         insured_relationship     0.348241
16            incident_severity    -0.296291
18               incident_state     0.283268
25      police_report_available     0.243387
37          

### Giải thích:

Phương pháp **SelectFromModel** từ thư viện sklearn.feature_selection. Phương pháp này hoạt động bằng cách sử dụng một mô hình học máy để đánh giá độ quan trọng của từng đặc trưng và sau đó chọn những đặc trưng quan trọng nhất.

   Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
  
       #### 1. LogisticRegression:
       Các đặc trưng được chọn:
* Index(['age', 'policy_csl', 'insured_education_level', 'insured_occupation',
       'incident_severity', 'incident_state', 'incident_city',
       'incident_hour_of_the_day', 'property_damage', 'witnesses',
       'property_claim', 'vehicle_claim', 'auto_make'],
      dtype='object')
      
   #### 2. LinearDiscriminantAnalysis:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip', 'insured_sex',
       'incident_severity', 'incident_hour_of_the_day', 'property_damage',
       'total_claim_amount', 'injury_claim', 'property_claim',
       'vehicle_claim'],
      dtype='object')
      
   #### 3. DecisionTreeClassifier:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip',
       'insured_occupation', 'insured_hobbies', 'incident_severity',
       'incident_location', 'police_report_available', 'total_claim_amount',
       'injury_claim', 'month_policy_bind_date', 'day_incident_date'],
      dtype='object')

In [19]:
def feature_selection_inRFE(model,name):
    rfe=RFE(estimator=model, n_features_to_select=10)
    fit=rfe.fit(x_train, y_train)
#     print("Num Features: %d"% fit.n_features_)
#     print("Selected Features: %s" % fit.support_)
#     print("Features Ranking: %s" %fit.ranking_)
#     for index, value in enumerate(fit.ranking_):
#         print(f"{value}: {x_train.columns[index]}")
    sorted_columns = [x for _, x in sorted(zip(fit.ranking_, x_train.columns))]
    sort=sorted(fit.ranking_)

#     for i,j in zip(sort,sorted_columns):
#         print("Columns sorted by RFE ranking",i,":",j)

    return [j for i,j in zip(sort,sorted_columns) if i==1]

In [20]:
models = [
    LogisticRegression(),
    LinearDiscriminantAnalysis(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LGBMClassifier(),  
    XGBClassifier(use_label_encoder=False)  
]
feature_RFE={}

names = ['LR', 'LDA', 'DTC', 'RD', 'LGBM', 'XGB']
for model,name in zip(models,names):
    print("RFE")
    print(f"Kiểm tra mô hình {name}")
    model.fit(x_train,y_train)
    print(feature_selection_inRFE(model,name))
    feature_RFE[name]=feature_selection_inRFE(model,name)

RFE
Kiểm tra mô hình LR
['age', 'authorities_contacted', 'auto_make', 'bodily_injuries', 'capital-loss', 'insured_sex', 'policy_annual_premium', 'policy_csl', 'policy_state', 'property_claim']
RFE
Kiểm tra mô hình LDA
['day_policy_bind_date', 'incident_severity', 'incident_type', 'insured_sex', 'month_policy_bind_date', 'number_of_vehicles_involved', 'policy_annual_premium', 'policy_state', 'property_damage', 'umbrella_limit']
RFE
Kiểm tra mô hình DTC
['authorities_contacted', 'auto_make', 'auto_model', 'auto_year', 'capital-loss', 'day_policy_bind_date', 'incident_hour_of_the_day', 'injury_claim', 'police_report_available', 'vehicle_claim']
RFE
Kiểm tra mô hình RD
['auto_make', 'auto_model', 'capital-gains', 'capital-loss', 'day_policy_bind_date', 'incident_severity', 'injury_claim', 'policy_annual_premium', 'total_claim_amount', 'year_policy_bind_date']
RFE
Kiểm tra mô hình LGBM
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing

### Giải thích:
**Recursive Feature Elimination (RFE)** là một phương pháp trong lựa chọn đặc trưng (feature selection) được sử dụng trong học máy để giảm số lượng đặc trưng của mô hình mà vẫn giữ được hiệu suất tốt.

Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
   * **'RD'**: RandomForestClassifier

In [21]:
special_models=[KNeighborsClassifier(),GaussianNB(),MLPClassifier(),GradientBoostingClassifier(),MultinomialNB(),SVC()]
names=['KNC','GNB','NN','BG','NB','SVM']
def feature_selection_inKBest(model, name):
    k_best = SelectKBest(score_func=f_classif, k=11)
    fit = k_best.fit(x_train, y_train)
    selected_features = fit.get_support()
    selected_column_names = x_train.columns[selected_features]
    return selected_column_names
feature_Kbest={}
for model,name in zip(special_models,names):
    print(f'Lua chon dac trung {name}')
    print(feature_selection_inKBest(model, name))
    feature_Kbest[name]=list(feature_selection_inKBest(model, name))

Lua chon dac trung KNC
Index(['policy_state', 'policy_csl', 'policy_annual_premium', 'umbrella_limit', 'insured_sex', 'capital-loss', 'incident_severity', 'incident_hour_of_the_day', 'injury_claim', 'auto_make', 'auto_model'], dtype='object')
Lua chon dac trung GNB
Index(['policy_state', 'policy_csl', 'policy_annual_premium', 'umbrella_limit', 'insured_sex', 'capital-loss', 'incident_severity', 'incident_hour_of_the_day', 'injury_claim', 'auto_make', 'auto_model'], dtype='object')
Lua chon dac trung NN
Index(['policy_state', 'policy_csl', 'policy_annual_premium', 'umbrella_limit', 'insured_sex', 'capital-loss', 'incident_severity', 'incident_hour_of_the_day', 'injury_claim', 'auto_make', 'auto_model'], dtype='object')
Lua chon dac trung BG
Index(['policy_state', 'policy_csl', 'policy_annual_premium', 'umbrella_limit', 'insured_sex', 'capital-loss', 'incident_severity', 'incident_hour_of_the_day', 'injury_claim', 'auto_make', 'auto_model'], dtype='object')
Lua chon dac trung NB
Index(['

### Giải thích:
Phương pháp **SelectKBest** từ thư viện sklearn.feature_selection để lựa chọn đặc trưng dựa trên kỹ thuật kiểm định thống kê.
 
Các model được dùng trong phương pháp này:
   * **'KNC'**: KNeighborsClassifier
   * **'GNB'**: GaussianNB
   * **'NN'**: MLPClassifier
   * **'BG'**: GradientBoostingClassifier
   * **'NB'**: MultinomialNB
   * **'SVM'**: SVC

In [22]:
print(feature_Kbest)
print(feature_SelectFromModel)
print(feature_RFE)

{'KNC': ['policy_state', 'policy_csl', 'policy_annual_premium', 'umbrella_limit', 'insured_sex', 'capital-loss', 'incident_severity', 'incident_hour_of_the_day', 'injury_claim', 'auto_make', 'auto_model'], 'GNB': ['policy_state', 'policy_csl', 'policy_annual_premium', 'umbrella_limit', 'insured_sex', 'capital-loss', 'incident_severity', 'incident_hour_of_the_day', 'injury_claim', 'auto_make', 'auto_model'], 'NN': ['policy_state', 'policy_csl', 'policy_annual_premium', 'umbrella_limit', 'insured_sex', 'capital-loss', 'incident_severity', 'incident_hour_of_the_day', 'injury_claim', 'auto_make', 'auto_model'], 'BG': ['policy_state', 'policy_csl', 'policy_annual_premium', 'umbrella_limit', 'insured_sex', 'capital-loss', 'incident_severity', 'incident_hour_of_the_day', 'injury_claim', 'auto_make', 'auto_model'], 'NB': ['policy_state', 'policy_csl', 'policy_annual_premium', 'umbrella_limit', 'insured_sex', 'capital-loss', 'incident_severity', 'incident_hour_of_the_day', 'injury_claim', 'auto

In [23]:
feature_model_selection={**feature_RFE,**feature_Kbest}
np.savez(f'{save_dir}/feature_model_selection.npz', feature_model_selection = feature_model_selection)
feature_model_selection

{'LR': ['age',
  'authorities_contacted',
  'auto_make',
  'bodily_injuries',
  'capital-loss',
  'insured_sex',
  'policy_annual_premium',
  'policy_csl',
  'policy_state',
  'property_claim'],
 'LDA': ['day_policy_bind_date',
  'incident_severity',
  'incident_type',
  'insured_sex',
  'month_policy_bind_date',
  'number_of_vehicles_involved',
  'policy_annual_premium',
  'policy_state',
  'property_damage',
  'umbrella_limit'],
 'DTC': ['authorities_contacted',
  'auto_make',
  'auto_model',
  'auto_year',
  'capital-loss',
  'day_policy_bind_date',
  'month_policy_bind_date',
  'police_report_available',
  'vehicle_claim',
  'year_policy_bind_date'],
 'RD': ['auto_make',
  'capital-gains',
  'capital-loss',
  'incident_severity',
  'injury_claim',
  'insured_education_level',
  'insured_sex',
  'policy_annual_premium',
  'property_claim',
  'year_policy_bind_date'],
 'LGBM': ['auto_make',
  'auto_model',
  'auto_year',
  'day_incident_date',
  'day_policy_bind_date',
  'month_incid

* gom nhóm thông tin về các đặc trưng được chọn từ hai phương pháp lựa chọn đặc trưng khác nhau (RFE và SelectKBest) vào từ điển **feature_model_selection**.

In [24]:
dict(np.load(f'{save_dir}/feature_model_selection.npz',allow_pickle=True))['feature_model_selection']

array({'LR': ['age', 'authorities_contacted', 'auto_make', 'bodily_injuries', 'capital-loss', 'insured_sex', 'policy_annual_premium', 'policy_csl', 'policy_state', 'property_claim'], 'LDA': ['day_policy_bind_date', 'incident_severity', 'incident_type', 'insured_sex', 'month_policy_bind_date', 'number_of_vehicles_involved', 'policy_annual_premium', 'policy_state', 'property_damage', 'umbrella_limit'], 'DTC': ['authorities_contacted', 'auto_make', 'auto_model', 'auto_year', 'capital-loss', 'day_policy_bind_date', 'month_policy_bind_date', 'police_report_available', 'vehicle_claim', 'year_policy_bind_date'], 'RD': ['auto_make', 'capital-gains', 'capital-loss', 'incident_severity', 'injury_claim', 'insured_education_level', 'insured_sex', 'policy_annual_premium', 'property_claim', 'year_policy_bind_date'], 'LGBM': ['auto_make', 'auto_model', 'auto_year', 'day_incident_date', 'day_policy_bind_date', 'month_incident_date', 'month_policy_bind_date', 'vehicle_claim', 'year_incident_date', 'yea